In [33]:
var Augur = require('augur.js');
var augur = new Augur();

var myConnectionInfo;

var httpAddress = "https://mainnet.infura.io/v3/f565925a6c5143188b5589a37b28139b"
//var httpAddress = "https://rinkeby.augur.net/ethereum-http"

var ethereumNode = { 
  httpAddresses: [
    httpAddress,
  ]
};

var augurNode = "ws://127.0.0.1:9001" // local Augur Node
//var augurNode = "ws://18.208.37.82:9001" // Augur Node on my aws machine

var finalizedMarketInfo = [];
var allMarketPriceHistory = [];
var outcomeBound = 2;


var marketInfoList = finalizedMarketInfo; // get all markets

function getPriceHistory(marketInfo){
    if (marketInfo[0].finalizationBlockNumber != null && 
        marketInfo[0].numOutcomes >= outcomeBound && 
        marketInfo[0].consensus['isInvalid'] == false) {
        //finalizedMarketInfo.push(marketInfo[0]);
        augur.markets.getMarketPriceHistory({marketId: marketInfo[0].id}, function(error, result) {
            allMarketPriceHistory.push([marketInfo[0], result])
        });
    }
}


function getFinalizedMarkets (markets, outcomeBound){
    var thisID;
    
    for (var i = 0; i < markets.length; i++) { 
        thisID = markets[i];
        augur.markets.getMarketsInfo({marketIds: [thisID]}, function (error, result) {getPriceHistory(result)})
    }
}

function getMarkets (connectionInfo){
    var universe = connectionInfo.ethereumNode.contracts.Universe;
    
    augur.markets.getMarkets({universe: universe}, 
                             function (err, result) {getFinalizedMarkets(result, outcomeBound)})
}

augur.connect({ ethereumNode, augurNode }, function (err, connectionInfo) {getMarkets(connectionInfo)});


connecting to augur-node: ws://127.0.0.1:9001
connecting to ethereum-node: {"httpAddresses":["https://mainnet.infura.io/v3/f565925a6c5143188b5589a37b28139b"]}
websocket ws://127.0.0.1:9001 opened
connected to augur
connected to ethereum


[ethrpc] http-transport unexpected status code IncomingMessage {
  _readableState:
   ReadableState {
     objectMode: false,
     highWaterMark: 16384,
     buffer: BufferList { head: null, tail: null, length: 0 },
     length: 0,
     pipes: null,
     pipesCount: 0,
     flowing: true,
     ended: true,
     endEmitted: true,
     reading: false,
     sync: true,
     needReadable: false,
     emittedReadable: false,
     readableListening: false,
     resumeScheduled: false,
     emitClose: true,
     destroyed: false,
     defaultEncoding: 'utf8',
     awaitDrain: 0,
     readingMore: true,
     decoder: null,
     encoding: null },
  readable: false,
  _events:
   [Object: null prototype] {
     end:
      [ [Function: responseOnEnd],
        [Function],
        { [Function: bound onceWrapper] listener: [Function] } ],
     close: [ [Function], [Function] ],
     data: [Function],
     error: [Function] },
  _eventsCount: 4,
  _maxListeners: undefined,
  socket:
   TLSSocket {
  

{ Error: Unexpected status code.
    at HttpTransport.<anonymous> (/Users/mettinger/node_modules/ethrpc/src/transport/http-transport.js:96:15)
    at Request.self.callback (/Users/mettinger/node_modules/ethrpc/node_modules/request/request.js:186:22)
    at Request.emit (events.js:182:13)
    at Request.<anonymous> (/Users/mettinger/node_modules/ethrpc/node_modules/request/request.js:1163:10)
    at Request.emit (events.js:182:13)
    at IncomingMessage.<anonymous> (/Users/mettinger/node_modules/ethrpc/node_modules/request/request.js:1085:12)
    at Object.onceWrapper (events.js:273:13)
    at IncomingMessage.emit (events.js:187:15)
    at endReadableNT (_stream_readable.js:1094:12)
    at process.internalTickCallback (internal/process/next_tick.js:72:19)
  code: 400,
  address:
   'https://mainnet.infura.io/v3/f565925a6c5143188b5589a37b28139b',
  data:
   { id: 16,
     jsonrpc: '2.0',
     method: 'eth_subscribe',
     params: [ 'newHeads', {} ] } }


In [35]:
allMarketPriceHistory.length

820

In [36]:
//const fs = require('fs');
var util = require('util');

function replacer(key, value) {
  if (value === false || value === true || value === null) {
    return JSON.stringify(value);
  }
  return value;
}

//fs.writeFileSync('./finalizedPrices.txt', util.inspect(allMarketPriceHistory) , 'utf-8');
fs.writeFileSync('./finalizedPrices.txt', JSON.stringify(allMarketPriceHistory, replacer));